In [ ]:
{
 "cells": [

# Initialization

In [ ]:
# Initialize Libraries
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import os
!pip install -U sentence_transformers


In [ ]:
# To save and read data files from your Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Make Preprocessor

# Load and explore the data set

In [ ]:
# Import the news.tsv file
path = '/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project4Again/'
news_df = pd.read_csv(path+'news.tsv', sep='\t', header = None)
news_df.columns = ['News_ID', 'Category', 'Subcategory', 'Title', 'Abstract', 'URL', 'Title_entities', 'Abstract_entities']
news_df.info()
news_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51283 entries, 0 to 51282
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   News_ID            51283 non-null  object
 1   Category           51283 non-null  object
 2   Subcategory        51283 non-null  object
 3   Title              51283 non-null  object
 4   Abstract           48617 non-null  object
 5   URL                51283 non-null  object
 6   Title_entities     51280 non-null  object
 7   Abstract_entities  51279 non-null  object
dtypes: object(8)
memory usage: 3.1+ MB


,News_ID,Category,Subcategory,Title,Abstract,URL,Title_entities,Abstract_entities
0,News_ID,Category,Subcategory,Title,Abstract,URL,Title_entities,Abstract_entities
1,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
2,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
3,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
4,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."


In [ ]:
news_df.to_csv(path + 'news.tsv', sep="\t", index=False)

In [ ]:
# Import the behavior.tsv file
behaviors_df = pd.read_csv(path+'behaviors.tsv', sep='\t', header = None)
behaviors_df.columns = ['Impression_ID', 'User_ID','Time', 'History','Impressions']
behaviors_df.info()
behaviors_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156965 entries, 0 to 156964
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Impression_ID  156965 non-null  int64 
 1   User_ID        156965 non-null  object
 2   Time           156965 non-null  object
 3   History        153727 non-null  object
 4   Impressions    156965 non-null  object
dtypes: int64(1), object(4)
memory usage: 6.0+ MB


,Impression_ID,User_ID,Time,History,Impressions
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...


# Feature Engineering

In [ ]:
# Make a dictionary for the text
news_text_dict = {
    row['News_ID']: f"{row['Title']} {row['Abstract']} {row['Category']} {row['Subcategory']}"
    for _, row in news_df.iterrows()
}
# news_text_dict

In [ ]:
# For the behavior file
behaviors_df = behaviors_df.dropna(subset=["History", "Impressions"])

# Make feature table with user, last_news, recommendation, and clicked
behavior_table_rows = []

for index, row in behaviors_df.iterrows():
  for impression in row['Impressions'].split():
    id = row['User_ID']
    last_news = row['History'].split()[-1]
    recommendation = impression.split('-')[0]
    clicked = impression.split('-')[1]
    behavior_table_rows.append([id, last_news, recommendation, clicked])

behavior_table = pd.DataFrame(behavior_table_rows, columns=['id', 'last_news', 'recommendation', 'clicked'])
behavior_table.head()

,id,last_news,recommendation,clicked
0,U13740,N31801,N55689,1
1,U13740,N31801,N35729,0
2,U91836,N25785,N20678,0
3,U91836,N25785,N39317,0
4,U91836,N25785,N58114,0


In [ ]:
# Add the text to the feature table
behavior_table["last_news_text"] = behavior_table["last_news"].map(news_text_dict)
behavior_table["recommendation_text"] = behavior_table["recommendation"].map(news_text_dict)
behavior_table.head()

,id,last_news,recommendation,clicked,last_news_text,recommendation_text
0,U13740,N31801,N55689,1,Joe Biden reportedly denied Communion at a Sou...,"Charles Rogers, former Michigan State football..."
1,U13740,N31801,N35729,0,Joe Biden reportedly denied Communion at a Sou...,Porsche launches into second story of New Jers...
2,U91836,N25785,N20678,0,OJ Simpson suing Vegas hotel for defamation ov...,Bode Miller delivered his twin boys after midw...
3,U91836,N25785,N39317,0,OJ Simpson suing Vegas hotel for defamation ov...,"Senior Trump official embellished résumé, had ..."
4,U91836,N25785,N58114,0,OJ Simpson suing Vegas hotel for defamation ov...,2020 Ford Explorer launch hardly went accordin...


# Create Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# For the behavior file
last_news_list = behavior_table['last_news_text'].astype(str).tolist()
recommend = behavior_table['recommendation_text'].astype(str).tolist()

news_embeddings_path = path + 'last_news_embeddings.npy'
recommend_embeddings_path = path + 'recommend_embeddings.npy'

# If embeddings exist, load them. If they don't, make them.
if os.path.exists(news_embeddings_path):
  last_news_embeddings = np.load(news_embeddings_path)
  del last_news_list
  gc.collect()
else:
  last_news_embeddings = model.encode(last_news_list, batch_size=32, show_progress_bar=True)
  np.save(news_embeddings_path, last_news_embeddings)

# If embeddings exist, load them. If they don't, make them.
if os.path.exists(recommend_embeddings_path):
  recommend_embeddings = np.load(recommend_embeddings_path)
  del recommend
  gc.collect()
else:
  recommend_embeddings = model.encode(recommend, batch_size=32, show_progress_bar=True)
  np.save(recommend_embeddings_path, recommend_embeddings)

last_news_embeddings.shape
recommend_embeddings.shape

Batches:   0%|          | 0/178844 [00:00<?, ?it/s]

Batches:   0%|          | 0/178844 [00:00<?, ?it/s]

(5723002, 384)

# Determine Scores!

In [ ]:
from numpy.linalg import norm

behavior_scored_path = path + 'behavior_scored.pkl'

# Cosine Similarity
if os.path.exists(behavior_scored_path):
    behavior_table = pd.read_pickle(behavior_scored_path)
else:
    dot_products = np.sum(last_news_embeddings * recommend_embeddings, axis=1)
    norms = norm(last_news_embeddings, axis=1) * norm(recommend_embeddings, axis=1)
    cosine_scores = dot_products / norms

    behavior_table["cosine_score"] = cosine_scores
    behavior_table.to_pickle(behavior_scored_path)

behavior_table["clicked"] = behavior_table["clicked"].astype(int)

In [ ]:
behavior_table

,id,last_news,recommendation,clicked,last_news_text,recommendation_text,cosine_score
0,U13740,N31801,N55689,1,Joe Biden reportedly denied Communion at a Sou...,"Charles Rogers, former Michigan State football...",0.139860
1,U13740,N31801,N35729,0,Joe Biden reportedly denied Communion at a Sou...,Porsche launches into second story of New Jers...,0.068852
2,U91836,N25785,N20678,0,OJ Simpson suing Vegas hotel for defamation ov...,Bode Miller delivered his twin boys after midw...,0.191428
3,U91836,N25785,N39317,0,OJ Simpson suing Vegas hotel for defamation ov...,"Senior Trump official embellished résumé, had ...",0.311390
4,U91836,N25785,N58114,0,OJ Simpson suing Vegas hotel for defamation ov...,2020 Ford Explorer launch hardly went accordin...,0.087033
...,...,...,...,...,...,...,...
5722997,U44625,N37863,N39317,0,"1 dead, several ill after chemical accident at...","Senior Trump official embellished résumé, had ...",0.111283
5722998,U64800,N48742,N61233,0,"Before his execution, a death row inmate told ...",Nor'easter to grind at US East Coast this week...,-0.046618
5722999,U64800,N48742,N33828,1,"Before his execution, a death row inmate told ...",Dwayne Johnson Announces 'Black Adam' 2021 Rel...,-0.012810
5723000,U64800,N48742,N19661,0,"Before his execution, a death row inmate told ...","Judge calls USC dad a 'thief,' gives longest p...",0.267542


In [ ]:
# Precision @ n and MRR function
def precision_and_mrr(df, n):
    clicked = 0
    counted = 0
    scores = []
    for index, group in df.groupby("id"):  # id = user_id
        top_n = group.sort_values("cosine_score", ascending=False).head(n)
        if top_n["clicked"].sum() > 0:
            clicked += 1
        counted += 1

        sorted = group.sort_values("cosine_score", ascending=False).reset_index(drop=True)
        for index_2, row in sorted.iterrows():
          if row["clicked"] == 1:
            scores.append(1 / (index_2 + 1))
            break
        else:
          scores.append[0]

    return clicked / counted if counted > 0 else 0, sum(scores) / len(scores)


n=10
precision, mrr = precision_and_mrr(behavior_table, n)
print(f"Precision @ {n} : {precision:.4f}")
print(f"MRR : {mrr:.4f}")

Precision @ 10 : 0.5910
MRR : 0.2753


In [ ]:
 ],
 "metadata": {},
 "nbformat": 4,
 "nbformat_minor": 0
}
